# Runpod

https://runpod.io/

**RunPod**는 고성능 GPU와 CPU 자원을 클라우드에서 저렴하게 임대하여 사용할 수 있는 서비스이다.  
개인용 컴퓨터에 고가의 그래픽카드가 없어도, RunPod를 이용하면 대형 AI 모델 학습, 추론, 데이터 분석 등 GPU가 필요한 작업을 쉽게 수행할 수 있다.

- **저렴한 가격**: 약 300만 원 상당의 GPU(예: RTX 3090)를 시간당 약 500원 수준으로 임대할 수 있다.
- **간편한 사용법**: 회원가입 후 원하는 GPU와 환경(Community Cloud 또는 Secure Cloud)을 선택하고, 몇 번의 클릭만으로 서버를 배포(Deploy)할 수 있다.
- **유연한 환경**: Jupyter Notebook, SSH, VSCode 등 다양한 방법으로 서버에 접속하여 개발할 수 있다.
- **다양한 활용**: LLM(대형 언어 모델) 파인튜닝, Stable Diffusion 등 AI/머신러닝 실험, 모델 배포 등 다양한 목적으로 활용된다.

## 1. Container / Volume / Network Disk 구분

**Container Disk**
- 컨테이너 디스크은 Pod가 실행될 때 생성되며, 운영체제와 임시 파일을 저장하는 공간이다.
- Pod의 라이프사이클에 직접적으로 연결되어 있어, Pod가 중지되거나 삭제되면 이 볼륨의 데이터도 함께 사라진다.
- 주로 임시 연산, 캐시, 로그 등 휘발성 데이터 저장에 사용한다.

**Volumn Disk**
- 볼륨 디스크은 Pod의 임대 기간 동안 지속되는 영구 저장소 역할을 한다.
- Pod가 중지되거나 재시작해도 데이터가 유지되지만, Pod 자체가 삭제되면 데이터도 함께 삭제된다.
- 기본적으로 `/workspace` 경로에 마운트되어 사용된다.
- 대용량 데이터, 학습 데이터, 결과물 등 장기 저장이 필요한 데이터에 적합하다.

**Network Disk**
- 네트워크 디스크은 Pod의 라이프사이클과 독립적으로 존재하는 영구 저장소다.
- 여러 Pod에 동시에 연결하여 데이터 공유가 가능하며, Pod를 삭제해도 데이터는 유지된다.
- 네트워크 볼륨은 Pod 생성 시에만 연결할 수 있으며, 한 번 연결하면 Pod가 삭제될 때까지 해제할 수 없다.
- 대규모 데이터셋 공유, 여러 작업 환경 간 데이터 이동 등에 적합하다.
- 저장소 성능과 비용 측면에서도 디스크 볼륨보다 유리할 수 있다.

**정리 표**

| 구분             | 데이터 지속성        | 공유 가능성 | 용도/특징                          | 기본 마운트 경로 |
|------------------|---------------------|-------------|-------------------------------------|------------------|
| Container Volume | Pod 종료 시 삭제    | 불가        | 임시 저장(운영체제, 캐시 등)         | `/`              |
| Disk Volume      | Pod 삭제 시 삭제    | 불가        | 영구 저장(학습 데이터, 결과 등)      | `/workspace`     |
| Network Volume   | Pod와 무관하게 유지 | 가능        | 여러 Pod 간 데이터 공유/영구 저장    | `/workspace`     |

## Community / Security Cloud

**Secure Cloud (보안 클라우드)**
- Runpod의 신뢰할 수 있는 파트너(T3/T4 데이터센터)에서 운영하는 인프라로, 고신뢰성, 중복성, 빠른 장애 대응, 강화된 보안이 특징이다.
- 민감하거나 기업용 워크로드에 적합하며, 데이터 보호와 프라이버시가 최우선으로 보장된다.
- 데이터는 격리된 환경(VPC)에서 동작하며, 암호화(전송/저장 모두), 다중 인증, 세분화된 권한 관리 등 다양한 보안 기능이 제공된다.

**Community Cloud (커뮤니티 클라우드)**
- 전 세계 다양한 개인/사업자 GPU 자원을 P2P 방식으로 연결하는 분산형 클라우드 플랫폼이다.
- 초대 기반으로 검증된 호스트만 참여 가능하며, Runpod의 기준을 충족해야 한다.
- 인프라 중복성(전원, 네트워크 등)은 Secure Cloud에 비해 낮지만, 합리적인 가격과 다양한 서버 옵션이 장점이다.
- 데이터는 멀티 테넌트 환경에서 격리되어 운영되며, 호스트가 데이터에 접근하는 것은 금지되어 있다.

**정리 표**

| 구분             | 인프라 위치/운영         | 보안 수준   | 가격/성능      | 추천 용도                  |
|------------------|-------------------------|-------------|----------------|----------------------------|
| Secure Cloud     | 신뢰 파트너 데이터센터   | 매우 높음   | 다소 높음      | 민감/기업용, 보안 필수      |
| Community Cloud  | 전 세계 분산 P2P 서버   | 표준 수준   | 저렴/다양      | 일반 연구, 비용 민감 워크로드|

## Runpod의 On-Demand와 Spot 인스턴스 차이

### 1. On-Demand 인스턴스

- **즉시 사용 가능**하며, 필요할 때 바로 생성하여 사용할 수 있다.
- **중단되지 않음**: 사용자가 종료하거나 삭제하지 않는 한, 인스턴스가 임의로 중단되지 않는다.
- **비용**: Spot 인스턴스에 비해 **비용이 높다**. 안정성과 지속성을 보장하기 때문에 더 높은 요금이 부과된다.
- **적합한 용도**:
  - 실시간 개발, 대화형 세션, 데모, 프로덕션 서비스 등 **중단이 허용되지 않는 작업**에 적합하다.
  - 예: Jupyter 노트북 실습, 실시간 서비스, 중요한 모델 튜닝 등.

### 2. Spot 인스턴스

- **여유 자원(Spare Capacity)**을 경매 방식으로 저렴하게 임대하는 방식이다.
- **중단 가능성**: 더 높은 우선순위의 작업(예: On-Demand 요청)이 들어오면 **언제든지 중단될 수 있다**. Runpod에서는 중단 신호(SIGTERM)가 5초 전에 전달되고, 이후 강제 종료(SIGKILL)된다.
- **비용**: On-Demand에 비해 **최대 90%까지 저렴**할 수 있다. 실제로 Runpod에서는 50% 이하로도 제공되는 경우가 많다.
- **적합한 용도**:
  - **중단에 유연한 작업**(stateless, 체크포인트 저장이 가능한 작업)에 적합하다.
  - 예: 대규모 배치 작업, 데이터 분석, 머신러닝/딥러닝 모델 학습, 렌더링, CI/CD 테스트 등.
  - 장기 실행 작업의 경우, 체크포인트를 주기적으로 저장하면 중단 시 재시작이 용이하다.

### 3. 주요 차이점 비교

| 항목                  | On-Demand 인스턴스                | Spot 인스턴스                        |
|----------------------|-----------------------------------|--------------------------------------|
| 가격                 | 높음                              | 매우 저렴(최대 90% 할인)             |
| 중단 가능성          | 없음                              | 있음(언제든 중단될 수 있음)          |
| 사용 가능성          | 항상 가능(자원이 있을 때)         | 여유 자원 있을 때만 가능             |
| 구매 방식            | 즉시 구매, 고정 가격              | 경매 방식, 변동 가격                 |
| 적합한 작업          | 실시간/중요/중단 불가 작업        | 배치/학습/중단 허용 작업             |
| 데이터 보존          | 직접 관리                         | 중단 전 5초간 데이터 저장 가능       |

### 4. 선택 가이드

- **On-Demand**: 중단이 허용되지 않는 실시간 서비스, 개발, 데모 등에 추천한다.
- **Spot**: 비용이 중요한 대규모 연산, 일시적·반복적 작업, 체크포인트가 가능한 모델 학습 등에 추천한다.

### 5. 활용 팁

- Spot 인스턴스는 비용 절감에 매우 효과적이지만, **중단을 대비한 자동 저장(체크포인트) 및 재시작 로직** 구현이 필수적이다.
- 개발·디버깅·실시간 서비스는 On-Demand, 대량 학습·배치 작업은 Spot을 혼합해 사용하는 전략이 효율적이다.

요약하면, On-Demand는 안정성과 지속성이 필요한 작업에, Spot은 비용 절감이 중요한 유연한 작업에 적합하다. Spot 인스턴스는 중단 위험이 있으므로, 데이터 손실 방지와 작업 자동 재시작을 반드시 고려해야 한다.